# 2. Hands-on experiment 2: k-means Clustering by Semidefinite Programming

Clustering is an unsupervised machine learning problem in which we try to partition a given data set into $k$ subsets based on distance between data points or similarity among them. The goal is to find $k$ centers and to assign each data point to one of the centers such that the sum of the square distances between them are minimal [1]. This problem is known to be NP-hard. 

### Clustering problem
Given a set of $n$ points in a $d-$dimensional Euclidean space, denoted by
\begin{equation*}
S = \{ \mathbf{s}_i = (s_{i1}, \cdots, s_{id})^\top~\in \mathbb{R}^d ~~ i = 1, \cdots, n\}
\end{equation*}
find an assignment of the $n$ points into $k$ disjoint clusters $\mathcal{S} = (S_1, \cdots, S_k)$ whose centers are $\mathbf{c}_j(j = 1, \cdots, k)$ based on the total sum of squared Euclidean distances from each point $\mathbf{s}_i$ to its assigned cluster centroid $\mathbf{c}_i$, i.e.,
\begin{equation}
f(S,\mathcal{S}) = \sum_{j=1}^k\sum_{i=1}^{|S_j|}\|\mathbf{s}_i^{j} - \mathbf{c}_j \|^2,
\label{eq:kmeans}\tag{Problem 1}
\end{equation}
where $|S_j|$ is the number of points in $S_j$, and $\mathbf{s}_i^{j} $ is the $i^{th}$ point in $S_j$.


In [5]:
from lib.part2.helpers import *
from scipy.sparse import isspmatrix
from lib.part2.Llyod_kmeans import *
import sys
sys.path.insert(0, "lib/part2")
from plotter import plot_func, plot_comp
# fix the seed
random.seed( 3 )

In [6]:
# Load data
Problem = sio.loadmat('lib/part2/data/clustering_data.mat')
C = np.double(Problem['C']) # euclidean distance matrix
N = int(Problem['N']) # number of data points
k = Problem['k'] # number of clusters
opt_val = Problem['opt_val'] # optimum value 
images = Problem['images'] # images for visualization after clustering
labels = Problem['labels'] # true labels
digits = Problem['digits'] # input data points

### Problem 2.1 Conditional gradient method for clustering fashion-mnist Data
---
#### <span style="font-variant:small-caps;">(e) *(20 points)*</span>


 $\triangleright$ Complete the missing lines in the function definitions of `HCGM` and `PDHG`, which implements Homotopy CGM and Vu-Condat algorithms, respectively. Run both methods $2000$ iterations to solve the $k$-means clustering problem.

$\triangleright$ Plot the convergence results of both algorithms using `plot_comp` function.

---


### Define operators
We provide 2 operators and their conjugates:
1. `A1`: Linear operator that takes the row sums
2. `At2`: Conjugate of operator A1
3. `A2`: Linear operator that takes the column sums 
4. `At2`: Conjugate of operator A2

In [3]:
A1 = lambda x: np.sum(x, axis = 1)
At1 = lambda y: np.transpose(np.matlib.repmat(y, N, 1))
A2 = lambda x: np.sum(x, axis = 0)
At2 = lambda y: (np.matlib.repmat(y, N, 1))

b = np.double(np.ones(N))

### Algorithm 1. Homotopy CGM

**Remark:** For simplicity, there is only one penalty parameter $\beta_k$ in the HCGM Algorithm. However, in practice, one can have different penalty parameters for different constraints. In our case, we advise you to ***multiply by 1000 the term $ (x_k - \text{proj}_{\mathcal{K}}(x_k))$** in Algorithm 1, in order to obtain a better practical convergence. This basically corresponds to having different penalty parameters for different constraints.


In [ ]:
def HCGM(kappa=10, maxit=np.int(1e3), beta0=1):
    # Initialize
    X = np.zeros((N,N))
    AX1_b = 0.0
    
    feasibility1 = [] # norm(A1(X)-b1)/norm(b1)
    feasibility2 = [] # dist(X, \mathcal{K})
    objective    = [] # f(x)
    cur_iter    = []   
    t    = []         # for time tracking
    
    iter_track = np.unique(np.ceil(np.power(2, np.linspace(0,20,50))))
    
    start = time.time()
    
    for iteration in range(1, maxit+1):
        
        # Update Step Size
        gamma = ??
        
        # Update beta
        beta_ = ??
        
        # Write down the vk to use in the lmo (eigenvalue routine)
        vk = ??? 
        vk = 0.5*(vk + vk.T)
        
        # Linear minimization oracle
        q, u = eigsh(???, k=1, tol=1e-16, which='SA')
        
        if ???:
            u = ??
        else:
            u = sqrt(kappa)*u
        
        X_bar = np.outer(u,u)
        
        # Obtain A*Xbar - b
        AX_bar_b = A1(X_bar)-b
        
        # Update A*X - b
        AX1_b = (1.0-gamma)*AX1_b + gamma*(AX_bar_b)
        
        # Update X
        X = ???
                
        if any(iteration == iter_track) or iteration==maxit:
            feasibility1.append(np.linalg.norm(AX1_b)/N)
            feasibility2.append(np.linalg.norm(np.minimum(X,0), ord='fro'))
            objective.append(np.sum(C.flatten()*X.flatten()))
            cur_iter.append(iteration)
            t.append(time.time()-start)
            print('{:03d} | {:.4e}| {:.4e}| {:.4e}|'.format(iteration, feasibility1[-1], feasibility2[-1],objective[-1]))
            
    return X, feasibility1, feasibility2, objective, cur_iter, t

#### Run HCGM

In [ ]:
X_HCGM, f1_HCGM, f2_HCGM, obj_HCGM, iter_HCGM, time_HCGM = HCGM(10, int(5000), 1)

### Algorithm 2. Vu-Condat

**Remarks:** 

- A similar observation applies tor the Vu-Condat algorithm: it is possible to use different dual step sizes $\{ \sigma_1 , \sigma_2, \ldots \}$. In our case, we advise you to **multiply the step-size for $y_3$ by $10^4$** to obtain a better practical convergence. (You can directly use the tuned stepsizes for PDHG.)

- In this part, you will need the projection operator of the nuclear norm ball. So, you can use what you implemented in part1 of the homework as the projection operator in the algorithm. 

In [ ]:
def PDHG(kappa=10, maxit=np.int(1e3), beta0=1):
    # Initialize
    X = np.zeros((N,N))
    Xprev = X
    AX1_b = 0.0
    
    y1 = A1(X)
    y2 = A2(X)
    y3 = X
    
    normC = np.linalg.norm(C,'fro')
    
    
    feasibility1 = [] # norm(A1(X)-b1)/norm(b1)
    feasibility2 = [] # dist(X, \mathcal{K})
    objective    = [] # f(x)
    cur_iter    = [] 
    t    = [] 
    
    L = 1e2      
    
    iter_track = np.unique(np.ceil(np.power(2, np.linspace(0,20,50))))
    
    start = time.time()
    # Primal and dual step sizes
    tau = 1/L # primal stepsize
    sigma = 1/(L**2*tau) # dual step size
    sigma2 = sigma*1e4   # different dual step size to use for the update of y3 for better practical performance.

    
    for iteration in range(1, maxit+1):
        
        # Primal variable update
        Xprev = X # store the previous iterate for reflection
        X = ???
        
        # Dual variable updates
        Xhat = ??? # the point at which the dual gradient is calculated
        y1 = ???
        y2 = ???
        y3 = ???
                
        # Update A*X - b
        AX1_b = A1(X)-b
        
        if any(iteration == iter_track) or iteration==maxit:
            feasibility1.append(np.linalg.norm(AX1_b)/N)
            feasibility2.append(np.linalg.norm(np.minimum(X,0), ord='fro'))
            objective.append(np.sum(C.flatten()*X.flatten()))
            cur_iter.append(iteration)
            t.append(time.time()-start)
            print('{:03d} | {:.4e}| {:.4e}| {:.4e}|'.format(iteration, feasibility1[-1], feasibility2[-1],objective[-1]))
            
    return X, feasibility1, feasibility2, objective, cur_iter, t

#### Run Vu-Condat

In [ ]:
X_PDHG, f1_PDHG, f2_PDHG, obj_PDHG, iter_PDHG, time_PDHG = PDHG(10, np.int(1000), 1, opt_val)

### Visualize the results

In [ ]:
iters = (iter_PDHG, iter_HCGM)
times = (time_PDHG, time_HCGM)
feas1 = (f1_PDHG, f1_HCGM)
feas2 = (f2_PDHG, f2_HCGM)
obj   = (obj_PDHG, obj_HCGM)
plot_comp(times, feas1,feas2, obj, opt_val)

### Rounding: Get the assignments from the result of the SDP
Getting the assignments requires going back to the $10$ dimensional space discussed before, and using the coordinates multiplied with the obtained matrix to construct a "denoised" version of the data points. This allows then to find the clusters from these $10$ dimensional data. See [3] for more details. Our implementation is the python reimplementation of their matlab code which can be found on [github](https://github.com/solevillar/kmeans_sdp).

In [ ]:
center_HCGM, assign_HCGM = sdp_rounding(X_HCGM,10, digits)
center_PDHG, assign_PDHG = sdp_rounding(X_PDHG,10, digits)

### k-means value: HCGM & Vu-Condat

In [ ]:
k_means_before = value_kmeans(digits, labels-1) # k_means value with true labels
k_means_after_HCGM  = value_kmeans(digits, assign_HCGM) # k_means value with assigned lables
k_means_after_PDHG  = value_kmeans(digits, assign_PDHG) # k_means value with assigned lables


print('k-means value initial: {:.4f}'.format(k_means_before))
print('k-means value for HCGM: {:.4f}'.format(k_means_after_HCGM))
print('k-means value for Vu-Condat: {:.4f}'.format(k_means_after_PDHG))

###  k-means value: Lloyd's algorithm

Run the Lloyd's algorithm directly on the input digits. 

In [ ]:
centers_Lloyd, classifications_Lloyd, k_means_Lloyd = kmeans(digits.T, 10)#k_means value with Lloyds k-means algorithm

print('k-means value for Lloyd''s algorithm: {:.4f}'.format(k_means_Lloyd))

## Questions

$\triangleright$ What are the final objective values? Are they below the optimal value provided to you? If yes, explain the reason. Answer in the box below.

$\triangleright$ Using the function `value_kmeans`, compute and report the $k$-means value before and after running both algorithms. 

$\triangleright$ Run the function `kmeans` a few times and report the $k$-means value obtained by Llyod's algorithm. Compare it with the ones obtained by rounding the solution of convex methods `HCGM` and `PDHG`. Comment on the result. (Write in the box below)

(<span style="font-variant:small-caps;"> Hint: </span> Note that when $\mathcal{X}$ is as given in (Problem 1), $\kappa u u^\top \in \text{lmo}_{\mathcal{X}}(X)$, where $u$ is the eigenvector corresponding to the smallest eigenvalue of $X$.)

# OPTIONAL: Additional results for clustering fMNIST Data

### Misclassification rates: HCGM & Vu-Condat

In [ ]:
print('Misclassification rate for HCGM: {:.4f}'.format(misclassification_rate(assign_HCGM, labels)))
print('Misclassification rate for Vu-Condat: {:.4f}'.format(misclassification_rate(assign_PDHG, labels)))

### Visualize samples and predicted labels

In [ ]:
classes = ['T-shirt/top','Trouser','Pullover','Dress','Coat','Sandal','Shirt','Sneaker','Bag','Ankle boot']

In [ ]:
def vis_samples(assignment, images, labels):
    assignment=assignment.astype(int)
    classes = ['T-shirt/top','Trouser','Pullover','Dress','Coat','Sandal','Shirt','Sneaker','Bag','Ankle boot']
    labels = labels-1
    rand_samp = np.random.randint(0,1000,25)
    plt.figure(figsize=(7,7))
    for i,samp in enumerate(rand_samp):
        plt.subplot(5,5,i+1)
        plt.imshow(1-np.reshape(images[samp],[28,28]), cmap=plt.cm.gray)
        plt.title('Pred. {0}\n Orig. {1}'.format(classes[assignment[samp].item()],classes[labels[samp].item()]))
        plt.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
vis_samples(assign_HCGM, images,labels)

In [ ]:
vis_samples(assign_PDHG, images,labels)